In checking writing systems, it's crucial to prevent alteration of check amounts. One common security method requires that the amount be written in numbers and spelled out in words as well. Create a dictionary that maps numbers to their corresponding word equivalents. Write a code that inputs a numeric check amount that is no more than 1000 and use the dictionary to write the word equivalent to the amount. For example, the amount 112.43 should be written as 

 ONE HUNDRED TWELVE AND 43/100

<font color='orange'><b><u>Make sure you use all features learned so far and if this code can be solved by creating your own class (this might not be any classical example for a class, but in the name of practicing, I ask you to), do so. At this point explanation of your solution is a must on top of comments in your code.</u></b></font>

In [1]:
import os

In [2]:
class Helper(object):

    @staticmethod
    def file_exists(file_path):
        """
        File exists check
        :param file_path: String full path to the file
        :return: Boolean value indicating if the file exists or not
        """
        return os.path.exists(file_path)
    
    @staticmethod
    def write_file(file_path, file_text):
        """
        Write file to disk
        :param file_path: String full path to the file
        :param file_text: String value to write to the file
        :return: Boolean value indicating the sucess of writting to the file
        """
        try:
            with open(file_path, 'w') as fh:
                fh.write(file_text)
        except IOError as e:
            print(f'Check the file path, {e.filename} could not be written to.')
            return False
        except TypeError as e:
            print(f'Check the type value of the file_text value: {file_text}')
            return False
        except:
            print('Generic write error')
            return False
        return True
                
    def read_file(self, file_path):
        """
        Read file from disk
        :param file_path: String Full path to the file
        :return: String value of the file contents or empty if file read fails
        """
        if not self.file_exists(file_path):
            print(f'Read file {file_path} does not exist')
            return ''
        try:
            with open(file_path, 'r') as fh:
                file_text = fh.read()
        except IOError as e:
            print(f'Check the file path, {e.filename} could not be read.')
            return ''
        except:
            print('Generic write error')
            return ''        
        return file_text
    
    @staticmethod
    def validated_input(input_prompt='Enter your name:', target_type=str, valid_options=[], 
                       valid_upper_bound=None):
        """
        Validated input wrapper method
        :param input_prompt: String used for the input prompt
        :param target_type: Python Type expected for the user input
        :param valid_options: List containing valid input options, defaults to empty list and is not applied
        :param valid_upper_bound: Maxium value allowed, defaults to None and is not applied
        :return: Typed of validated user input
        """
        # Confirm that target_type is actually a Type
        if not isinstance(target_type, type):
            print(f'Invalid target_type value {target_type}, not a Python Type')
        # Loop forever waiting for the user to enter the correctly formatted input
        while True:
            # Get the user input
            user_input = input(input_prompt)
            try:
                # Try casting the user input to the target type
                user_input_typed = target_type(user_input)
                # If there is a list of valid options enforce user input membership
                if len(valid_options) > 0 and user_input_typed not in valid_options:
                    valid_options_str = ', '.join([str(vo) for vo in valid_options])
                    print(f'Invalid input, must be one of the following values: {valid_options_str}')
                    continue
                # Apply the upper bound max check on int and float types if specified
                if ((target_type==int and isinstance(valid_upper_bound, int)) or 
                    (target_type==float and isinstance(valid_upper_bound, float))):
                    if user_input_typed > valid_upper_bound:
                        print(f'Invalid input, must be less the valid upper bound {valid_upper_bound}')
                        continue
                break
            except:
                print(f'Invalid input {user_input} is not of type {target_type.__name__}')
        return user_input_typed

<hr />

My strategy to solve this problem is to start with a simple number to words dictionary and then expand it by 
combining the simple words, ultimately resulting in a numbers to words dictionary covering the necessary 
combinations.

I then compute the check amount from the dollar amount by checking the length of the string to identify the 
amount of digits, which are then used to lookup the word parts.

I made some minor changes to the validated_input() helper to include a valid upper bound parameter and 
return the typed user input.

In [3]:
# Starter word tokens
numbers_to_words = {1: 'ONE', 2: 'TWO', 3: 'THREE', 4: 'FOUR', 5: 'FIVE', 6: 'SIX', 7: 'SEVEN', 8: 'EIGHT',
                    9: 'NINE', 10: 'TEN', 11: 'ELEVEN', 12: 'TWELEVE', 13: 'THIRTEEN', 14: 'FOURTEEN', 
                    15: 'FIFTEEN', 16: 'SIXTEEN', 17: 'SEVENTEEN', 18: 'EIGHTEEN', 19: 'NINETEEN', 
                    1000: 'ONE THOUSAND'}

In [4]:
roots = {2: 'TWENTY', 3: 'THIRTY', 4: 'FOURTY', 5: 'FIFTY', 6: 'SIXTY', 7: 'SEVNTY', 8: 'EIGHTY', 9: 'NINETY'}
for number in range(20, 100):
    # Exclude the roots themselves (nothing divisable by 10 evenly)
    if number % 10 != 0:
        # Create composite word - root + number's word token and add to the dict
        word = roots.get(int(str(number)[0])) + numbers_to_words.get(int(str(number)[1]))
        numbers_to_words[number] = word

In [5]:
# Add the roots as wholes, multiply by 10 to bring them to the correct level
for number, word in roots.items():
    numbers_to_words[number*10] = word

In [6]:
# Add the hundreds
for number in range(1, 10):
    # Create composite word - number word root + HUNDRED and add to the dict
    word = f'{numbers_to_words.get(number)} HUNDRED'
    numbers_to_words[number*100] = word

In [7]:
def compute_check_amount(dollar_amount):
    """
    Compute the verbose check amount from a given dollar amount
    :param dollar_amount: Float Dollar amount value
    :return String of verbose check amount
    """
    # Get the dollars and cents componts of the float
    dollars, cents = divmod(dollar_amount, 1)
    dollars = int(dollars)
    w = 'ZERO'
    if len(str(dollars)) > 1 and len(str(dollars)) < 3:
        # Process a 2 digit dollar amount
        w = numbers_to_words.get(dollars)
    elif len(str(dollars)) == 3:
        # Process 3 digit dollar amount in two parts - hundreds and the rest
        w1 = numbers_to_words.get(int(str(dollars)[0:1]) * 100)
        if dollars % 100 == 0:
            # Handle even hundreds
            w = w1
        else:
            w2 = numbers_to_words.get(int(str(dollars)[1:3]))
            w = f'{w1} {w2}'
    elif len(str(dollars)) == 4:
        # Process 4 digit dollar amount in three parts - thousands, hundreds and the rest
        w1 = numbers_to_words.get(int(str(dollars)[0:1]) * 1000)
        if x % 1000 == 0:
            # Handle even thousands
            w = w1
        elif x % 100 == 0: 
            # Handle even hundreds
            w2 = numbers_to_words.get(int(str(dollars)[1:2]) * 100)
            w = f'{w1} {w2}'
        else:
            w3 = numbers_to_words.get(int(str(dollars)[2:4]))
            w = f'{w1} {w2} {w3}'    
    # Final dollars and cents string generation
    return f'{w} DOLLARS AND {round(cents,2)*100:.0f}/100'

In [8]:
hlp = Helper()
dollar_amount = hlp.validated_input('Enter a dollar amount', target_type=float, valid_upper_bound=1000.)
print(compute_check_amount(dollar_amount=dollar_amount))

Enter a dollar amount112.43
ONE HUNDRED TWELEVE DOLLARS AND 43/100
